<a href="https://colab.research.google.com/github/satyamsingh1004/ML/blob/master/PredictTrafficVolume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

import datetime

In [0]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
#print(train.shape, train.head(), train.describe(),train.info())

In [0]:
train['is_holiday'] = train['is_holiday'].apply(lambda x: 0 if x == 'None' else 1)
test['is_holiday'] = test['is_holiday'].apply(lambda x: 0 if x == 'None' else 1)

In [0]:
train['weather'] = train['weather_type'].str.lower() + ' ' + train['weather_description'].str.lower()
train = pd.concat([train,pd.get_dummies(train['weather'], prefix='weather',dummy_na=True)],axis=1)
train = train.drop(['weather_type', 'weather_description','weather'], axis = 1)

test['weather'] = test['weather_type'].str.lower() + ' ' + test['weather_description'].str.lower()
test = pd.concat([test,pd.get_dummies(test['weather'], prefix='weather',dummy_na=True)],axis=1)
test = test.drop(['weather_type', 'weather_description','weather'], axis = 1)

In [12]:
type(train.columns)

pandas.core.indexes.base.Index

In [0]:
y = train['traffic_volume']
x = train.drop(['date_time', 'traffic_volume'], axis = 1)

test_dateTime = test['date_time']
test_x = test.drop(['date_time'], axis = 1)

In [0]:
from sklearn import preprocessing

x = x.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x = pd.DataFrame(x_scaled)

test_x = test_x.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(test_x)
test_x = pd.DataFrame(x_scaled)


In [0]:
train_x, val_x, train_y, val_y = train_test_split(x,y, test_size=0.1, random_state=1)

In [8]:
model = RandomForestRegressor(random_state=1) #1569.3175703703703
#model = DecisionTreeRegressor(random_state=1) # 1880.1973333333333
#model = xgb.XGBRegressor() # 1692.4393237847223
#model.fit(train_x, train_y)
model.fit(x, y)
#preds = model.predict(val_x)
#print(mean_absolute_error(val_y, preds))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [10]:
preds = model.predict(test_x)

submission = pd.DataFrame({'date_time':test_dateTime, 'traffic_volume':preds})
submission = submission[['date_time', 'traffic_volume']]
submission.head()
submission.to_csv("submission.csv", index=False)

ValueError: ignored